In [1]:
import findspark
findspark.init()

In [2]:
# Create SparkSession and import all required packages
from pyspark.sql import SparkSession,types

spark = SparkSession.builder.master("local").appName('Json File')\
                    .getOrCreate()

In [3]:
#read multiline JSON file
input_df=spark.read.json('input2.json', multiLine=True)

In [4]:
input_df.printSchema()

root
 |-- Education: struct (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
#read multiline JSON file
input_df1=spark.read.json('input1.json', multiLine=True)

In [6]:
input_df1.printSchema()

root
 |-- Education: struct (nullable = true)
 |    |-- Age: long (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [12]:
input_df.schema['Education']

StructField(Education,StructType(List(StructField(Qualification,StringType,true),StructField(year,LongType,true))),true)

In [13]:
from pyspark.sql.types import *
from pyspark.sql.functions import col,lit,struct

def flatten_struct(schema, prefix=""):
    result = []
    for elem in schema:
        if isinstance(elem.dataType, StructType):
            result += flatten_struct(elem.dataType, prefix + elem.name + ".")
           # result.replace("Column<b","")
        else:
            result.append(col(prefix + elem.name).alias(prefix + elem.name))
    return result

l1=flatten_struct(input_df.schema)
l2=flatten_struct(input_df1.schema)

In [15]:
l2

[Column<b'Education.Age AS `Education.Age`'>,
 Column<b'Education.Qualification AS `Education.Qualification`'>,
 Column<b'Education.year AS `Education.year`'>,
 Column<b'name AS `name`'>]

In [22]:
list1=[]
list2=[]
for i in l1:
    list1.append(str(i).split("`")[1])
    
for i in l2:
    list2.append(str(i).split("`")[1])
    

In [23]:
chk=set(list2)-set(list1)
chk

{'Education.Age'}

In [24]:
from pyspark.sql.types import *
chk=set(list2)-set(list1)
for i in chk:
    if i.find("."):
        colm=i.split(".")[0]
        colm_new=i.split(".")[1]
        s_fields = input_df.schema[colm].dataType.names
        s_type=input_df1.schema[colm].dataType[colm_new].dataType
                
        in_df=input_df.withColumn(colm,
                            struct(*([col(colm)[c].alias(c) for c in s_fields] +
                                     [lit("null").cast(s_type).alias(colm_new)]
                                     ))
                                 )
        s_fields = sorted(in_df.schema[colm].dataType.names)
        
        in_df=in_df.withColumn(colm,
                            struct(*([col(colm)[c].alias(c) for c in s_fields] ))
                                 )
        


In [25]:
in_df.printSchema()

root
 |-- Education: struct (nullable = false)
 |    |-- Age: long (nullable = true)
 |    |-- Qualification: string (nullable = true)
 |    |-- year: long (nullable = true)
 |-- name: string (nullable = true)



In [26]:
out_df=in_df.union(input_df1)

In [27]:
out_df.show()

+----------------+-------+
|       Education|   name|
+----------------+-------+
|    [, BE, 2011]| Clarke|
|    [, BE, 2010]|Michael|
|[28, BCOM, 2013]|   AZAR|
|  [35, BE, 2010]|   CHIN|
+----------------+-------+

